In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [5]:
db = pd.read_excel('Detections_of_IBC_2018_07_06.xlsx')

In [6]:
db_central = db[(db.Route == 'Central Mediterranean')]

In [7]:
db_central.drop(['BorderLocation', 'Route'], axis=1, inplace=True)
db_central.set_index('NationalityLong', inplace=True)

/srv/conda/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [8]:
cluster_now = db_central.iloc[:,-2:].sum(axis=1).sort_values(ascending=False)
cluster_before = db_central.iloc[:,-4:-2].sum(axis=1).sort_values(ascending=False)

In [15]:
top10_now = cluster_now[:10]
top10_before = cluster_before[:10]

In [31]:
index = 0
for n in top10_now.index:
    print(n)

Tunisia
Sudan
Eritrea
Nigeria
Côte d'Ivoire
Mali
Guinea
Algeria
Senegal
Pakistan


In [36]:
top_num = 0
for nat in top10_before.index:
    if top10_now.index[top_num] in top10_before.index:
        print(top10_now.index[top_num] + ' WAS in previous cluster.')
        #percentage_chage(top10_now.index[top_num])
    else:
        print(top10_now.index[top_num] + ' WAS NOT in previous cluster.')
    top_num += 1

Tunisia WAS in previous cluster.
Sudan WAS in previous cluster.
Eritrea WAS in previous cluster.
Nigeria WAS in previous cluster.
Côte d'Ivoire WAS in previous cluster.
Mali WAS in previous cluster.
Guinea WAS in previous cluster.
Algeria WAS NOT in previous cluster.
Senegal WAS NOT in previous cluster.
Pakistan WAS NOT in previous cluster.
